In [1]:
Verbose = False
if Verbose:
    def vprint(*args, **kwargs): print(*args, **kwargs, flush=True)
else: # do-nothing function
    def vprint(*args, **kwargs): None

In [2]:
import torch
import numpy as np
import pandas as pd
pd.set_option("display.width", 380)
pd.set_option('max_colwidth', 100)

import matplotlib.pyplot as plt

from IPython.display import display

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Load the sentences

In [41]:
df_train = pd.read_csv("./sick_train/SICK_train.txt", sep="\t")
df_train = df_train.drop(['relatedness_score'], axis=1)

df_dev = pd.read_csv("./sick_trial/SICK_trial.txt", sep="\t")
df_dev = df_dev.drop(['relatedness_score'], axis=1)

df_test = pd.read_csv("./sick_test/SICK_test.txt", sep="\t")
df_test = df_test.drop(['relatedness_score'], axis=1)

In [4]:
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [5]:
from gensim import corpora

In [6]:
class SickDataset(Dataset):
    endOfSentence   = '</s>'
    startOfSentence = '<s>'
    separator2Sentences = '<sep>'
    
    text_label = ["NEUTRAL", "ENTAILMENT", "CONTRADICTION"]
    
    tokens = [startOfSentence, separator2Sentences, endOfSentence]
    
    def join_sentence(self, row):
        """
        Create a new sentence (<s> + s_A + <sep> + s_B + </s>)
        """
        sentence_a = row['sentence_A'].lower().split(" ")
        sentence_b = row['sentence_B'].lower().split(" ")
        return np.concatenate((
            [self.startOfSentence],
            sentence_a,
            [self.separator2Sentences],
            sentence_b,
            [self.endOfSentence]
        ))
    
    def series_text_2_labelID(self, series, keep_n=1000):
        """
        Convert text Label into label id
        """
        reverse_dict = {v: k for k, v in  dict(enumerate(self.text_label)).items()}
        return series.map(reverse_dict)
    
    def series_2_dict(self, series, keep_n):
        """
        Convert document (a list of words) into a list of indexes
        AND apply some filter on the documents
        """
        dictionary = corpora.Dictionary(series)
        dictionary.filter_extremes(
            no_below=1,
            no_above=1,
            keep_n=keep_n,
            keep_tokens=self.tokens)
        return dictionary
    
    
    def __init__(self, df, vocabulary_size, dic=None):
        # Label text as ids
        df["entailment_id"] = self.series_text_2_labelID(df['entailment_judgment'])
        
        # Add <s>,</s>,<sep> tokens to the vocabulary
        df['sentence_AB'] = df.apply(self.join_sentence, axis=1)
        
        # check if the dictionary is given
        if dic is None:
            # Create the Dictionary
            self.dictionary = self.series_2_dict(df['sentence_AB'], vocabulary_size)
        else:
            self.dictionary = dic
        
        # sentence of words -> array of idx
        # Adds unknown to the voc (idx = len(dictionary)), len(dictionary) = vocabulary_size
        # Adds one to each (no tokens at 0, even <unk>)
        # 0 is for the padding when using mini-batch
        df["word_idx"] = df["sentence_AB"].apply(
            lambda word: np.array(self.dictionary.doc2idx(word, unknown_word_index=vocabulary_size)) + 1
        )
        
        self.df = df
        
    def getRef(self, index):
        return self.df['sentence_AB'][index]
        
    def __getitem__(self, index):
        return (
            self.df['word_idx'][index],
            self.df['entailment_id'][index])
    
    def getDictionary(self):
        return self.dictionary
    
    def __len__(self):
        return len(self.df)

    
vocabulary_size = 1500

# Dataset

In [7]:
# Create the train dataset
sick_dataset_train = SickDataset(df_train, vocabulary_size)
sick_dataset_train.df.head()

,pair_ID,sentence_A,sentence_B,entailment_judgment,entailment_id,sentence_AB,word_idx
0,1,A group of kids is playing in a yard and an old man is standing in the background,A group of boys in a yard is playing and a man is standing in the background,NEUTRAL,0,"[<s>, a, group, of, kids, is, playing, in, a, yard, and, an, old, man, is, standing, in, the, ba...","[2, 4, 9, 14, 12, 11, 16, 10, 4, 19, 6, 5, 15, 13, 11, 17, 10, 18, 7, 3, 4, 9, 14, 8, 10, 4, 19,..."
1,2,A group of children is playing in the house and there is no man standing in the background,A group of kids is playing in a yard and an old man is standing in the background,NEUTRAL,0,"[<s>, a, group, of, children, is, playing, in, the, house, and, there, is, no, man, standing, in...","[2, 4, 9, 14, 20, 11, 16, 10, 18, 21, 6, 23, 11, 22, 13, 17, 10, 18, 7, 3, 4, 9, 14, 12, 11, 16,..."
2,3,The young boys are playing outdoors and the man is smiling nearby,The kids are playing outdoors near a man with a smile,ENTAILMENT,1,"[<s>, the, young, boys, are, playing, outdoors, and, the, man, is, smiling, nearby, <sep>, the, ...","[2, 18, 30, 8, 24, 16, 26, 6, 18, 13, 11, 28, 1501, 3, 18, 12, 24, 16, 26, 25, 4, 13, 29, 4, 27, 1]"
3,5,The kids are playing outdoors near a man with a smile,A group of kids is playing in a yard and an old man is standing in the background,NEUTRAL,0,"[<s>, the, kids, are, playing, outdoors, near, a, man, with, a, smile, <sep>, a, group, of, kids...","[2, 18, 12, 24, 16, 26, 25, 4, 13, 29, 4, 27, 3, 4, 9, 14, 12, 11, 16, 10, 4, 19, 6, 5, 15, 13, ..."
4,9,The young boys are playing outdoors and the man is smiling nearby,A group of kids is playing in a yard and an old man is standing in the background,NEUTRAL,0,"[<s>, the, young, boys, are, playing, outdoors, and, the, man, is, smiling, nearby, <sep>, a, gr...","[2, 18, 30, 8, 24, 16, 26, 6, 18, 13, 11, 28, 1501, 3, 4, 9, 14, 12, 11, 16, 10, 4, 19, 6, 5, 15..."


In [8]:
# Create the dev dataset
dictionary_train = sick_dataset_train.getDictionary()

sick_dataset_dev = SickDataset(df_dev, vocabulary_size, dictionary_train)

In [9]:
# Create the test dataset
sick_dataset_test = SickDataset(df_test, vocabulary_size, dictionary_train)

In [10]:
pd.DataFrame(list(zip(sick_dataset_train.getRef(2), sick_dataset_train[2][0]))).T

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,<s>,the,young,boys,are,playing,outdoors,and,the,man,...,are,playing,outdoors,near,a,man,with,a,smile,</s>
1,2,18,30,8,24,16,26,6,18,13,...,24,16,26,25,4,13,29,4,27,1


# Pretrained embeddings
https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76  
For each word in dataset’s vocabulary, we check if it is on GloVe’s vocabulary. If it do it, we load its pre-trained word vector. Otherwise, we initialize a random vector.

In [ ]:
EMBEDDINGS_SIZE = 50

In [ ]:
from torchtext import vocab
#vocab is shared across all the text fields
#CAUTION: GloVe will download all embeddings locally (862 MB).
pretrained_emb = vocab.GloVe(name='6B', dim=EMBEDDINGS_SIZE)

# 0 is for the padding when using mini-batch (start at one, shift by one)
weights_matrix = np.zeros((vocabulary_size + 2, EMBEDDINGS_SIZE)) # do not forget the unk

# build a matrix of weights that will be loaded into the PyTorch embedding layer
for word_id in sick_dataset_train.dictionary:
    word = sick_dataset_train.dictionary[word_id]
    if word in pretrained_emb.stoi:
        pretrained_emb_ID = pretrained_emb.stoi[word]
        
        weights_matrix[word_id+1] = pretrained_emb.vectors[pretrained_emb_ID]
    else:
        weights_matrix[word_id+1] = np.random.normal(scale=0.6, size=(EMBEDDINGS_SIZE, ))
        
# UNK
weights_matrix[vocabulary_size+1] = np.random.normal(scale=0.6, size=(EMBEDDINGS_SIZE, ))
        
pretrained_emb_vec = torch.tensor(weights_matrix, dtype=torch.float32)
print("Downloaded: Pretained Embedding matrix: " +  str(pretrained_emb.vectors.size()))
print("Adapted:    Pretained Embedding matrix: " +  str(pretrained_emb_vec.size()))

In [ ]:
print(sick_dataset_train.dictionary.doc2idx(["the", "The"]))
print(sick_dataset_train.dictionary[17])
print(pretrained_emb_vec[17+1])
# Glove dim=50 word=the vector[:4] = 0.418 0.24968 -0.41242 0.1217

# To DataLoader

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
def pad_collate(batch):
    seqs_labels = np.array(batch)[:,1]
    
    vectorized_seqs = np.array(batch)[:,0]
    seq_lengths = torch.LongTensor([len(x) for x in vectorized_seqs])
    
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
        
    vectorized_seqs = np.array(seq_tensor)
    
    return torch.tensor(vectorized_seqs), torch.LongTensor([ x for x in seqs_labels])


BATCH_SIZE = 8

train_loader = DataLoader(dataset=sick_dataset_train,
                          batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate)

dev_loader = DataLoader(dataset=sick_dataset_dev,
                          batch_size=1, shuffle=False)

test_loader = DataLoader(dataset=sick_dataset_test,
                          batch_size=1, shuffle=False)

# Debug the padding
# display([ x for x in enumerate(train_loader)][0]) # has padding (sample of same size padded with 0)
# display([ x for x in enumerate(dev_loader)][0]) # no batch == no padding

# Evaluation helpers

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import precision_recall_fscore_support

import itertools

def confusion_scores(total_labels, total_pred):
    classes = SickDataset.text_label
    title='Confusion matrix'
    
    cm = confusion_matrix(total_labels, total_pred, labels=[0, 1, 2])
    
    plt.rcParams["figure.figsize"] = [6, 5]
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title, color='gray', fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, [c.lower() for c in classes], rotation=45 , style='italic', color='gray', fontsize=12)
    plt.yticks(tick_marks, [c.lower() for c in classes], color='gray', style='italic', fontsize=12)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', color='gray', fontsize=14)
    plt.xlabel('Predicted label', color='gray', fontsize=14)
    plt.tight_layout()


def evaluate(model, loader, whileTraining=False, criterion=None):
    """
    Displays the confusion_matrix the precision recall fscore
    If in whileTrainnig Mode only return the accuracy and loss
    """
    model.eval()
    with torch.no_grad():
        total_labels = []
        total_pred = []
        for _, (data, target) in enumerate(loader):

                data = data.to(device)
                target = target.to(device)

                output = rnn(data)

                # Get the Accuracy
                _, predicted = torch.max(output.data, dim=1)
                correct = (predicted == target).sum().item()
                
                total_labels.append(list(target.cpu().numpy()))
                total_pred.append(list(predicted.cpu().numpy()))
                
        model.train()
        if whileTraining and criterion!=None:
            loss = criterion(output, target)
            return ((accuracy_score(total_labels, total_pred) * 100), loss)

                
        confusion_scores(total_labels, total_pred)
        
        print("Accuracy:  {:.4f}".format(accuracy_score(total_labels, total_pred)))
        
        # compute per-label precisions, recalls, F1-scores, and supports instead of averaging 
        metrics = precision_recall_fscore_support(
                                        total_labels, total_pred,
                                        average=None, labels=[0, 1, 2])
        
        df = pd.DataFrame(list(metrics), index=['Precision', 'Recall', 'Fscore', 'support'],
                                   columns=SickDataset.text_label)
        df = df.drop(['support'], axis=0)
        display(df.T)
        


# Create the RNN Model

In [ ]:
from torch import nn

In [ ]:
class RNNClassifier(nn.Module):
    # Our model

    def __init__(self, input_voc_size, embedding_size, hidden_size):
        super(RNNClassifier, self).__init__()
        
        self.input_voc_size = input_voc_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.rnn_out_size = hidden_size * 2

        
        self.num_classes = 3
        
        # Add the padding token (0) (+1 to voc_size)
        # Pads the output with the embedding vector at padding_idx whenever it encounters the index..
        self.embedding = nn.Embedding(input_voc_size+1, embedding_size, padding_idx=0)
        # Load the pretrained embeddings
        #self.embedding.weight = nn.Parameter(pretrained_emb_vec) 
        # embeddings fine-tuning
        self.embedding.weight.requires_grad = False
        
        self.rnn = nn.GRU(
              input_size=embedding_size,
              hidden_size=hidden_size,
              batch_first=True,
              bidirectional=True,
        )
        
        self.fc1 = nn.Linear(self.rnn_out_size, self.num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    # input shape: B x S (input size)
    def forward(self, x):
        
        vprint("\nsize input", x.size())
        batch_size = x.size(0)
        
        # Initialize hidden (num_layers * num_directions, batch_size, hidden_size)
        h_0 = torch.zeros(2, batch_size, self.hidden_size)
        vprint("size hidden init", h_0.size())
        
        # When creating new variables inside a model (like the hidden state in an RNN/GRU/LSTM),
        # make sure to also move them to the device (GPU or CPU).
        h_0 = h_0.to(device)

        # Embedding B x S -> B x S x I (embedding size)
        emb = self.embedding(x)
        vprint("size Embedding", emb.size())
        
        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        out, hidden = self.rnn(emb, h_0)
        
        vprint("size hidden", hidden.size())
        
        rnn_out = torch.cat((hidden[0], hidden[1]), 1)
        vprint("size rnn out", rnn_out.size())
        
        
        # Use the last layer output as FC's input
        layout_fc1 = self.fc1(rnn_out)
        vprint("size layout fc1", layout_fc1.size())
        
        fc_output = self.softmax(layout_fc1)
        
        return fc_output    
    
# Add the unknown token (+1 to voc_size)
rnn = RNNClassifier(vocabulary_size+1, EMBEDDINGS_SIZE, 21)
rnn.to(device)
print(rnn)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
weights = [1-((sick_dataset_train.df['entailment_id'] == i).sum() / len(sick_dataset_train)) for i in range(3)]
class_weights = torch.FloatTensor(weights).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.002)

# Training the Model 

In [ ]:
%%time
# Train the model

num_epochs=35

train_loss = []

rnn.train()
for epoch in range(num_epochs):
    total_correct = 0
    total_target = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        output = rnn(data)
        
        vprint(output)
        loss = criterion(output, target)
        
        rnn.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Get the Accuracy
        _, predicted = torch.max(output.data, dim=1)
        correct = (predicted == target).sum().item()
        
        total_correct += correct
        total_target += target.size(0)
        
        if batch_idx % 100 == 0 or batch_idx % 100 == 1 or batch_idx == len(train_loader)-1:
            train_loss.append(loss.cpu().detach().numpy())
            print('\rEpoch [{:3}/{}] | Step [{:5}/{} ({:3.0f}%)] | Loss {:.3f} | Accuracy {:.2f}%'.format(
                    epoch+1, num_epochs,
                    batch_idx * len(data), 
                    len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), 
                    loss.item(), 
                    (total_correct / total_target) * 100), end=' ')
            
        if Verbose:
            break
            
    accuracy_dev, loss_dev = evaluate(rnn, dev_loader, criterion=criterion, whileTraining=True)
    print("@ Loss_dev {:.3f} | Accuracy_dev {:.2f}%".format(loss_dev, accuracy_dev))


print("Learning finished!")

In [ ]:
from scipy.signal import savgol_filter
import ipywidgets as widgets
from ipywidgets import interact

def update_losses(smooth=31):
    plt.rcParams["figure.figsize"] = [9, 7]
    plt.title("Loss Curves", color='gray', fontsize=20)
    plt.xlabel('Steps', fontsize=16, color='gray')
    plt.ylabel('Loss', fontsize=16, color='gray')
    _line, = plt.plot(train_loss)
    _line.set_ydata(savgol_filter(train_loss, smooth, 3))

interact(update_losses, smooth=(5, 201, 4));


# Evaluate the model on dev

In [ ]:
%%time

evaluate(rnn, dev_loader)

# Evaluate the model on test

In [ ]:
%%time

# evaluate(rnn, test_loader)

# Data augmentation
It needs to install first nltk (ask to add this in `requirement.txt`) and download it with `nltk.download('wordnet')`  just once (other libraries may come)

In [11]:
import re # for deleting words with regexp
import nltk
from nltk.corpus import wordnet
# nltk.download('wordnet') # Synonyms
# nltk.download('averaged_perceptron_tagger') #POS Tagger
# nltk.download("punkt") #word_tokenizer
# nltk.download('universal_tagset') #universal tagger

# Warning : 'wagon,' & 'brown,' hasn't be regularly tokenized 

### TODO
# Générer de nouvelles phrases en remplaçant les mots qui ont des synonymes par leur pos associé si possible
# Tester les nouvelles phrases avec le modèle du baseline

In [29]:
# The common POS taggers between nltk and Wordnet are ADJ, ADV, NOUN, VERB
# ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v' 
# ADJ_SAT (Adjective satellite) is an adjective that occurs regurlarly with nouns (e.g. : atomic bomb)
# It's a choice to combine ADJ and ADJ_SAT & to delete phrasals verbs (they're using '_')


# Mapper which takes a synset POS and converts into a nltk POS
def wordnetpos_2_ntlkpos(tag):
    if (tag.startswith('a') or tag.startswith('s')):
        return "ADJ"
    elif tag.startswith('r'):
        return "ADV"
    elif tag.startswith('n'):
        return "NOUN"
    elif tag.startswith('v'):
        return "VERB"

# Create a dictionary of synonyms
synonyms = []
for w in dictionary_train:
    synonyms.append(dictionary_train[w])
    for syn in wordnet.synsets(dictionary_train[w]):
        pos_tag = wordnetpos_2_ntlkpos(syn.pos())
        for lemma in syn.lemmas(): 
            synonyms.append((lemma.name().lower(),pos_tag))

# Make unique tuples without changing the order and delete the first element ('')
synonyms = [ element for (i,element) in enumerate(synonyms) if element not in synonyms[:i] ]
del synonyms[0]
# print(synonyms)


# Suppress phrasal verbs
# r = re.compile(".*_.*")
# newsynonyms = list(filter(r.match, synonyms))
# #print(newsynonyms)

# for word in synonyms:
#     if word in newsynonyms:
#         synonyms.remove(word)
# print(synonyms)


# Reorder results so we have a dictionary with {key : [(value, POS),(value, POS)]}
synonyms_dictionary = dict()
values = []

for word in synonyms:
    if isinstance(word, str):
        key = word
        if key not in synonyms_dictionary:
            synonyms_dictionary[key] = []
    elif isinstance(word, tuple):
        synonyms_dictionary[key].append(word)
    else: #debugging
        print("something is not good here")

# print(synonyms_dictionary)

In [35]:
# Tokenize and apply a POS tagger on sentences in order to find synonyms
pos_sentencesA = []
pos_sentencesB = []

for sentence in sick_dataset_train.df.sentence_A:
    tokens = nltk.word_tokenize(sentence.lower())
    tagged_sentence = nltk.pos_tag(tokens, tagset='universal')
    pos_sentencesA.append(tagged_sentence)
    
for sentence in sick_dataset_train.df.sentence_B:
    tokens = nltk.word_tokenize(sentence.lower())
    tagged_sentence = nltk.pos_tag(tokens, tagset='universal')
    pos_sentencesB.append(tagged_sentence)

# print(pos_sentencesA[0])

In [39]:
# Generate phrases from the dictionary
common_pos = ['ADJ', 'ADV', 'NOUN', 'VERB']
generated_phrases = []

# Generate a new word from its POS
# By ex., an ADV will be changed by another ADV and add then the rest of the words to complete the new sentence
# generated_phrases is like 
# [['a', 'group', 'of', 'kids'], ['a', 'round', 'of', 'kids'], ['a', 'group', 'of', 'children']]
for pos in common_pos:
    for sentence in pos_sentencesA:
        for idx,(key1,value1) in enumerate(sentence):
            if value1 == pos and key1 in synonyms_dictionary:
                synonyms_values = synonyms_dictionary.get(key1)
                for (key2, value2) in synonyms_values:
                    if value2 == pos:
                        generated_phrases.append(key2)
                    else:
                        generated_phrases.append(key1)
            else:
                generated_phrases.append(key1)
# print(generated_phrases)

# FIXME : Boucles sur la génération mal faites, recréer un tableau à chaque subsitution effectuée par exemple

In [ ]:
# Train with the generated sentences
# Converts sentences into dataframe and to dataloader